### This project features interactive maps and HTML markup. Since Github does not support dynamic displays for notebooks, it is necessary to download the project to render the elements and to manipulate the data visualizations.

# Data Preparation

This is the <b>first in a series of three Jupyter notebooks</b> on the 2018 Food Consumption and CO<sub>2</sub> Emissions. This activity is in partial fulfillment of the Tidy Tuesdays deliverables for probationary Lyrids of the <b>Center for Complexity and Emerging Technologies, College of Computer Studies, De La Salle University</b>.

<b>Climate change and global warming are pressing environmental issues &mdash; and among the foremost drivers of these are human-induced emissions of greenhouse gases, such as carbon dioxide (CO<sub>2</sub>)</b>. While this project does not seek to present a professional or rigorous statistical analysis, the author of this series of Jupyter notebooks would like to increase awareness on the importance of data-driven policy directions and to hopefully contribute to the present discourse on how food consumption can greatly impact our carbon footprint. 

<hr/>

The required dataset for this Tidy Tuesdays activity is the 2018 Food Consumption and CO<sub>2</sub> Emissions from the R Community's Tidy Tuesdays (Github): https://github.com/rfordatascience/tidytuesday/tree/master/data/2020/2020-02-18. 

To enrich the analysis and visualization, the following datasets were integrated:

- Country Codes and Names - http://country.io/names.json
- World Bank Country and Lending Groups (Classification for the 2018 Fiscal Year) - https://datahelpdesk.worldbank.org/knowledgebase/articles/906519-world-bank-country-and-lending-groups

These datasets are stored in the folder <code>data</code> of the repository.

# PRELIMINARIES

We will be using the following modules and libraries in our project:
- <code>re</code> - for performing operations involving regular expressions (https://docs.python.org/3/library/re.html)
- <code>json</code> - for working with JSON (JavaScript object notation) data (https://docs.python.org/3/library/json.html)
- <code>pandas</code> - for performing some data manipulation (https://pandas.pydata.org/)
- <code>numpy</code> - for performing efficient numerical analysis (https://numpy.org/)
- <code>scipy</code> - for carrying out scientific computing tasks (https://www.scipy.org/)
- <code>pywaffle</code> - for creating waffle charts (https://pywaffle.readthedocs.io/en/latest/)
- <code>matplotlib</code> - for generating data visualizations (https://matplotlib.org/)
- <code>plotly</code> - for generating data visualizations (https://plotly.com/)

Note that <code>plotly</code> is not automatically bundled with an Anaconda installation. To install this package with conda (https://anaconda.org/plotly/plotly), either of these commands has to be run:

<code>conda install -c plotly plotly</code> <br/>
<code>conda install -c plotly/label/test plotly</code>

The same goes for <code>pywaffle</code>. To install this package with conda, either of these commands has to be run:

<code>conda install -c conda-forge pywaffle</code> <br/>
<code>conda install -c conda-forge/label/cf201901 pywaffle</code> <br/>
<code>conda install -c conda-forge/label/cf202003 pywaffle</code> <br/>

In [1]:
import re
import json

import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from pywaffle import Waffle

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected = True)

The last two lines of code are for displaying the figures (generated via <code>plotly</code>) on the Jupyter notebook itself.

Moreover, we also suppress the warning generated by Pandas for chained assignments.

In [2]:
pd.options.mode.chained_assignment = None  

We set the number of rows to be displayed when the data are presented in tabular form via <code>pandas</code>. Note that, during actual data preparation and analysis, this is usually set to a high value (or the entire datasets themselves are printed) in order to view more rows, preliminarily identify issues related to formatting or encoding, and understand anomalous results. 

In [3]:
NUM_ROWS = 10

pd.set_option('display.max_rows', NUM_ROWS)
pd.set_option('display.min_rows', NUM_ROWS)

<hr/>

# DATA PREPARATION

Before we can perform any form of data analysis, we first have to load the dataset and perform some preliminary data cleaning. This is necessary in order to have an initial understanding of the nature and structure of the data at hand, standardize the formatting and encoding of entries, and improve the quality of the data before exploratory analysis is conducted. 

## A. Load the Dataset

The 2018 Food Consumption and CO2 Emissions from the R community's Tidy Tuesdays (Github) consists of a single CSV file. This dataset has four columns:
- <code>country</code>: name of the country or region
- <code>food_category</code>: food product
- <code>consumption</code>: food product consumption (including wastage) in kg/person/year
- <code>co2_emmission</code>: median carbon dioxide produced in kg CO<sub>2</sub>/person/year 

The methodological process for the construction of this dataset is explained by the German-based group nu3 (2018) in https://www.nu3.de/blogs/nutrition/food-carbon-footprint-index-2018.

In [4]:
data_raw = pd.read_csv('data/food_consumption.csv')

It is a good idea to manipulate a copy of the dataset to avoid accidental modification of the contents of the original.

In [5]:
data = data_raw.copy(deep = True)
data

,country,food_category,consumption,co2_emmission
0,Argentina,Pork,10.51,37.20
1,Argentina,Poultry,38.66,41.53
2,Argentina,Beef,55.48,1712.00
3,Argentina,Lamb & Goat,1.56,54.63
4,Argentina,Fish,4.36,6.96
...,...,...,...,...
1425,Bangladesh,Milk - inc. cheese,21.91,31.21
1426,Bangladesh,Wheat and Wheat Products,17.47,3.33
1427,Bangladesh,Rice,171.73,219.76
1428,Bangladesh,Soybeans,0.61,0.27


<hr/>

## B. Clean the Dataset

We now proceed to the process of data cleaning per se. In particular, null values, duplicate entries, improper data types, and inconsistent representations or values are handled in this step.

### 1. Checking of Size, Shape, and Data Types

Looking at the shape of the dataset, we find that there are 1430 rows and 4 columns, corresponding to the following details:
- name of the country or region
- food product
- food product consumption (including wastage) in kg/person/year
- median carbon dioxide in kg CO<sub>2</sub>/person/year

We also print the number of null values per column and the data types in order to identify areas that need to be addressed during data preparation.

In [6]:
print("Size/Shape: ", data.shape, "\n") 
print("Checking for null values:\n", data.isnull().sum(), "\n")
print("Data types:\n", data.dtypes)

Size/Shape:  (1430, 4) 

Checking for null values:
 country          0
food_category    0
consumption      0
co2_emmission    0
dtype: int64 

Data types:
 country           object
food_category     object
consumption      float64
co2_emmission    float64
dtype: object


As seen in the details above, there are no null values in the dataset. Moreover, all the data types are correct.

### 2. Removal of Duplicates

Since each entry is unique identified by the combination of the country and the food category, we discard duplicates since they are most likely instances of misencoding. 

In [7]:
data = data.drop_duplicates()

print("AFTER DROPPING DUPLICATES\n")
print("Size/Shape: ", data.shape, "\n") 
print("Checking for null values:\n", data.isnull().sum(), "\n")
print("Data types:\n", data.dtypes)

AFTER DROPPING DUPLICATES

Size/Shape:  (1430, 4) 

Checking for null values:
 country          0
food_category    0
consumption      0
co2_emmission    0
dtype: int64 

Data types:
 country           object
food_category     object
consumption      float64
co2_emmission    float64
dtype: object


Observe that the shape of the dataset did not change, evincing that the dataset does not contain any duplicates.

<hr/>

## C. Standardize Names and Labels

Going through the entries shows that the values and representations are consistent all throughout the dataset. Nevertheless, in the interest of data visualization, it may be helpful (1) to standardize the names of the countries to conform with those used in third-party map-generating libraries (e.g., <code>plotly</code>) and (2) to tweak the labels of the food categories to be more grammatically formal.

### 1. Standardization of Country Names

The following JSON file was consulted as reference in standardizing the names of countries: http://country.io/names.json. We load it onto our notebook via the <code>json</code> module:

In [8]:
with open('data/countries.json') as f:
    countries = json.load(f)
    
print(countries)

{'BD': 'Bangladesh', 'BE': 'Belgium', 'BF': 'Burkina Faso', 'BG': 'Bulgaria', 'BA': 'Bosnia and Herzegovina', 'BB': 'Barbados', 'WF': 'Wallis and Futuna', 'BL': 'Saint Barthelemy', 'BM': 'Bermuda', 'BN': 'Brunei', 'BO': 'Bolivia', 'BH': 'Bahrain', 'BI': 'Burundi', 'BJ': 'Benin', 'BT': 'Bhutan', 'JM': 'Jamaica', 'BV': 'Bouvet Island', 'BW': 'Botswana', 'WS': 'Samoa', 'BQ': 'Bonaire, Saint Eustatius and Saba ', 'BR': 'Brazil', 'BS': 'Bahamas', 'JE': 'Jersey', 'BY': 'Belarus', 'BZ': 'Belize', 'RU': 'Russia', 'RW': 'Rwanda', 'RS': 'Serbia', 'TL': 'East Timor', 'RE': 'Reunion', 'TM': 'Turkmenistan', 'TJ': 'Tajikistan', 'RO': 'Romania', 'TK': 'Tokelau', 'GW': 'Guinea-Bissau', 'GU': 'Guam', 'GT': 'Guatemala', 'GS': 'South Georgia and the South Sandwich Islands', 'GR': 'Greece', 'GQ': 'Equatorial Guinea', 'GP': 'Guadeloupe', 'JP': 'Japan', 'GY': 'Guyana', 'GG': 'Guernsey', 'GF': 'French Guiana', 'GE': 'Georgia', 'GD': 'Grenada', 'GB': 'United Kingdom', 'GA': 'Gabon', 'SV': 'El Salvador', 'GN':

To identify the countries whose names have to be standardized, we filter country names that are present in our CO<sub>2</sub> Emissions dataset but are not found in the reference JSON file. 

In [9]:
ref_countries = set(countries.values())
data_countries = set(data['country'])

print(data_countries - ref_countries)

{'USA', 'Hong Kong SAR. China', 'Taiwan. ROC', 'Congo'}


In this regard, a list, <code>standardized_names</code>, consisting of tuples is constructed. The first element in each tuple refers to the non-standardized name found in the datasets while the second element refers to the standardized one.

In [10]:
standardized_names = [("Taiwan. ROC", "Taiwan"),
                      ("USA", "United States"),
                      ("Hong Kong SAR. China", "Hong Kong"),
                      ("Congo", "Republic of the Congo")]

Note that Congo can be quite ambiguous, as it can refer to either the Democratic Republic of the Congo (with capital Kinshasa) and the Republic of the Congo (with capital Brazzaville). In this project, <code>Congo</code> is standardized to <code>Republic of the Congo</code> following the convention by the World Health Organization (n.d.): https://www.who.int/countries/cog/.

To verify that the names have been standardized, we print the (updated) unique countries.

In [11]:
for name in standardized_names:
    data['country'] = data['country'].str.replace(re.escape(name[0]), name[1])
   
print(data['country'].unique())

['Argentina' 'Australia' 'Albania' 'Iceland' 'New Zealand' 'United States'
 'Uruguay' 'Luxembourg' 'Brazil' 'Kazakhstan' 'Sweden' 'Bermuda' 'Denmark'
 'Finland' 'Ireland' 'Greece' 'France' 'Canada' 'Norway' 'Hong Kong'
 'French Polynesia' 'Israel' 'Switzerland' 'Netherlands' 'Kuwait'
 'United Kingdom' 'Austria' 'Oman' 'Italy' 'Bahamas' 'Portugal' 'Malta'
 'Armenia' 'Slovenia' 'Chile' 'Venezuela' 'Belgium' 'Germany' 'Russia'
 'Croatia' 'Belarus' 'Spain' 'Paraguay' 'New Caledonia' 'South Africa'
 'Barbados' 'Lithuania' 'Turkey' 'Estonia' 'Mexico' 'Costa Rica' 'Bolivia'
 'Ecuador' 'Panama' 'Czech Republic' 'Romania' 'Colombia' 'Maldives'
 'Cyprus' 'Serbia' 'United Arab Emirates' 'Algeria' 'Ukraine' 'Pakistan'
 'Swaziland' 'Latvia' 'Bosnia and Herzegovina' 'Fiji' 'South Korea'
 'Poland' 'Saudi Arabia' 'Botswana' 'Macedonia' 'Hungary'
 'Trinidad and Tobago' 'Tunisia' 'Egypt' 'Mauritius' 'Bulgaria' 'Morocco'
 'Slovakia' 'Niger' 'Kenya' 'Jordan' 'Japan' 'Georgia' 'Grenada'
 'El Salvador' 'Cub

### 2. Standardization of Food Categories

The dataset consists of 11 unique food categories, which are printed below:

In [12]:
print(data['food_category'].unique())

['Pork' 'Poultry' 'Beef' 'Lamb & Goat' 'Fish' 'Eggs' 'Milk - inc. cheese'
 'Wheat and Wheat Products' 'Rice' 'Soybeans' 'Nuts inc. Peanut Butter']


Tweaking these labels for grammatical formality and consistency, we replace instances of the conjunction <code>and</code> and the abbreviation <code>inc.</code> with an ampersand symbol <code>&</code>.

In [13]:
standardized_categories = [("Milk - inc. cheese", "Milk & Cheese"),
                           ("Wheat and Wheat Products", "Wheat & Wheat Products"),
                           ("Nuts inc. Peanut Butter", "Nuts & Peanut Butter")]

To verify that the labels have been standardized, we print the (updated) unique food categories.

In [14]:
for category in standardized_categories:
    data['food_category'] = data['food_category'].str.replace(re.escape(category[0]), category[1])
    
print(data['food_category'].unique())

['Pork' 'Poultry' 'Beef' 'Lamb & Goat' 'Fish' 'Eggs' 'Milk & Cheese'
 'Wheat & Wheat Products' 'Rice' 'Soybeans' 'Nuts & Peanut Butter']


## *One Final Check*

As a final sanity check, we print the cleaned dataset.

In [15]:
data

,country,food_category,consumption,co2_emmission
0,Argentina,Pork,10.51,37.20
1,Argentina,Poultry,38.66,41.53
2,Argentina,Beef,55.48,1712.00
3,Argentina,Lamb & Goat,1.56,54.63
4,Argentina,Fish,4.36,6.96
...,...,...,...,...
1425,Bangladesh,Milk & Cheese,21.91,31.21
1426,Bangladesh,Wheat & Wheat Products,17.47,3.33
1427,Bangladesh,Rice,171.73,219.76
1428,Bangladesh,Soybeans,0.61,0.27


# References

- European Union. (2021). *Animal products*. https://ec.europa.eu/info/food-farming-fisheries/animals-and-animal-products/animal-products/
- Mottaleb, K., Rahut, D.N., Kruseman, G., & Erenstein, O. (2017). Wheat production and consumption dynamics in an Asian rice economy: The Bangladesh case. *European Journal of Development Research, 30*(1), 1-24. doi:10.1057/s41287-017-0096-1
- nu3. (2018). *Food carbon footprint index 2018*. https://www.nu3.de/blogs/nutrition/food-carbon-footprint-index-2018
- Quinton, A. (2019, June 27). *Cows and climate change*. University of California, Davis. https://www.ucdavis.edu/food/news/making-cattle-more-sustainable
- Stylianou, N., Guibourg, C., & Briggs, H. (2019, August 9). *Climate change calculator: What's your diet's carbon footprint?* https://www.bbc.com/news/science-environment-46459714
- World Health Organization. (n.d.). *Congo*. https://www.who.int/countries/cog/